In [2]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import requests
import xml.etree.ElementTree as ET
import json
import pyprind
import os, sys
from sklearn.utils import resample
# Ignore specific Pandas warnings
warnings.filterwarnings("ignore")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
pip install --upgrade numpy scipy


In [ ]:
dir_tree = 'C:\\Users\\sunitha\\Downloads\\dt'
for dir_path, dir_names, file_names in os.walk(dir_tree):
    for file_name in file_names:
        try:
            os.rename(os.path.join(dir_path, file_name), os.path.join(dir_tree, file_name))
        except OSError:
            print ("Could not move %s " % os.join(dir_path, file_name))
df = pd.read_csv('C:\\Users\\sunitha\\Downloads\\dt\\spotify_data.csv', encoding='ISO-8859-1')
# Verify relevant columns
print(df.columns)

### Build an artist table with file,title, artist columns

In [ ]:
# Build an artist table with 'track_id', 'artists', and 'track_name' columns
def make_artist_table(df):
    # Create a new DataFrame with selected columns
    artist_table = df[['track_id', 'artists', 'track_name']].copy()
    
    # Rename columns for consistency if needed
    artist_table.columns = ['track_id', 'artist', 'title']
    
    # Ensure no missing values in the key columns
    artist_table.dropna(subset=['track_id', 'artist', 'title'], inplace=True)
    
    return artist_table

# Use the function to create the artist table
artist_table = make_artist_table(df)
# Display the final artist table
print(artist_table.head())

                 track_id             artist  \
0  4VXIryQMWpIdGgYR4TrjT1         Juice WRLD   
1  0UsmyJDsst2xhX1ZiFF3JW  Schoolgirl Byebye   
2  285pBltuF7vW8TeWk8hdRR         Juice WRLD   
3  4fbwTO3DJ2qryMddov9RbB      Fleetwood Mac   
4  0rKtyWc8bvkriBthvHKY8d               Joji   

                                          title  
0                        All Girls Are The Same  
1                                     Year,2015  
2                                  Lucid Dreams  
3  Rhiannon (Will You Ever Win) - 2018 Remaster  
4                      SLOW DANCING IN THE DARK  


### Add the lyrics column

In [ ]:
df['lyrics'] = pd.Series('', index=df.index)

### download the PyLyrics package to download lyrics from the website

In [ ]:
import nest_asyncio
import asyncio
import aiohttp
import pyprind

# Apply nest_asyncio to allow running asyncio within Jupyter/other event loops
nest_asyncio.apply()

# Caching lyrics responses to avoid redundant API calls
lyrics_cache = {}

# Asynchronous function to fetch lyrics for a single song using aiohttp
async def fetch_lyrics(session, artist, title):
    if (artist, title) in lyrics_cache:
        return lyrics_cache[(artist, title)]  # Return from cache if exists
    
    url = f"https://api.lyrics.ovh/v1/{artist}/{title}"
    try:
        async with session.get(url) as response:
            if response.status == 200:
                data = await response.json()
                lyrics = data.get('lyrics', 'Lyrics not found.')
                lyrics_cache[(artist, title)] = lyrics  # Cache the result
                return lyrics
            else:
                return "Failed to fetch lyrics"
    
    except Exception as e:
        # Handle request exceptions
        return f"Error: {str(e)}"

# Asynchronous function to fetch lyrics for all songs
async def fetch_lyrics_for_all(df, max_concurrent_requests=10):
    df['lyrics'] = ''
    progress_bar = pyprind.ProgBar(len(df), monitor=True)
    semaphore = asyncio.Semaphore(max_concurrent_requests)

    async with aiohttp.ClientSession() as session:
        # Function to fetch lyrics with semaphore to control concurrency
        async def fetch_with_semaphore(row_id):
            async with semaphore:
                artist = df.loc[row_id, 'artists']
                title = df.loc[row_id, 'track_name']
                lyrics = await fetch_lyrics(session, artist, title)
                df.at[row_id, 'lyrics'] = lyrics
                progress_bar.update()
        
        # Create tasks and run them concurrently
        tasks = [fetch_with_semaphore(row_id) for row_id in df.index]
        await asyncio.gather(*tasks)

    return df

# Assuming 'df' is your DataFrame with 'artists' and 'track_name' columns
# Run the asynchronous function
df_with_lyrics = asyncio.run(fetch_lyrics_for_all(df))

# Display the DataFrame with fetched lyrics
print(df_with_lyrics[['artists', 'track_name', 'lyrics']].head())


0% [##############################] 100% | ETA: 00:00:00

             artists                                    track_name  \
0         Juice WRLD                        All Girls Are The Same   
1  Schoolgirl Byebye                                     Year,2015   
2         Juice WRLD                                  Lucid Dreams   
3      Fleetwood Mac  Rhiannon (Will You Ever Win) - 2018 Remaster   
4               Joji                      SLOW DANCING IN THE DARK   

                                              lyrics  
0  They're rotting my brain, love\r\nThese hoes a...  
1                             Failed to fetch lyrics  
2  I still see your shadows in my room\r\nCan't t...  
3                             Failed to fetch lyrics  
4  I don't want a friend (just me)\r\nI want my l...  



Total time elapsed: 00:05:52


### download lyrics with the arguments of artist and track name

In [ ]:
df_with_lyrics.to_csv("C:\\Users\\sunitha\\Downloads\\miniversionproject\\text mining on music lyrics\\data\\df_lyr_backup.csv", index=False)

### drop rows that has no lyrics

In [ ]:
df = df[df.lyrics!='']

### remove songs that is not English song, cited from https://github.com/rasbt/musicmood

In [ ]:
import nltk
nltk.download('words')  # Download the word list if you haven't done so

from nltk.corpus import words
english_vocab = set(words.words())

def eng_ratio(text):
    text_vocab = set(w.lower() for w in text.split() if w.lower().isalpha())
    if len(text_vocab) == 0:
        return 0  # Return 0 if there are no valid words
    unusual = text_vocab.difference(english_vocab)
    diff = len(unusual) / len(text_vocab)
    return diff

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\sunitha\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


### Till now, we got the songs with lyrics, but we have to tag each song with mood. Here I download the tags from Last.fm and classified each some with happy mood or sad mood

In [ ]:
import asyncio
import aiohttp
import pyprind
import pandas as pd
from urllib.parse import quote

# Function to fetch tags asynchronously
async def fetch_tags(session, artist, title):
    # Ensure the artist and title are strings
    artist = str(artist)
    title = str(title)
    
    artist_encoded = quote(artist)
    title_encoded = quote(title)
    
    url = f"http://ws.audioscrobbler.com/2.0/?method=track.getTopTags&api_key=1a66acac46e578280bf3c788a976a4ed&artist={artist_encoded}&track={title_encoded}&format=json"
    
    try:
        async with session.get(url) as response:
            if response.status == 200:
                content_type = response.headers.get("Content-Type", "").lower()
                if "application/json" in content_type:
                    result = await response.json()
                    tags = [tag['name'] for tag in result.get('toptags', {}).get('tag', [])]
                    return tags
            return []
    except Exception as e:
        print(f"Error fetching tags for {artist} - {title}: {e}")
        return []

# Function to fetch tags for all songs asynchronously
async def fetch_tags_for_songs(df):
    async with aiohttp.ClientSession() as session:
        # Create a list of tasks to fetch tags concurrently
        tasks = []
        for row_id in df.index:
            artist = df.loc[row_id]['artists']
            title = df.loc[row_id]['track_name']
            tasks.append(fetch_tags(session, artist, title))
        
        # Execute all tasks concurrently and wait for results
        tags_list = await asyncio.gather(*tasks)
        
        # Store the tags in the DataFrame
        df['tags'] = [', '.join(tags) if tags else 'No tags found' for tags in tags_list]
    
    return df

# Function to run the process
def fetch_tags_for_songs_with_progress(df):
    # Initialize progress bar
    pbar = pyprind.ProgBar(df.shape[0])
    
    # Fetch tags concurrently
    loop = asyncio.get_event_loop()
    df = loop.run_until_complete(fetch_tags_for_songs(df))
    
    # Update the progress bar after the operation
    pbar.update(df.shape[0])
    
    # Return the updated DataFrame
    return df

# Assuming 'df' is your DataFrame, initialize the 'tags' column
df['tags'] = ''

# Fetch tags with progress bar
df = fetch_tags_for_songs_with_progress(df)

# Display the final DataFrame
print(df[['artists', 'track_name', 'tags']])


0% [##############################] 100% | ETA: 00:00:00

                artists                                    track_name  \
0            Juice WRLD                        All Girls Are The Same   
1     Schoolgirl Byebye                                     Year,2015   
2            Juice WRLD                                  Lucid Dreams   
3         Fleetwood Mac  Rhiannon (Will You Ever Win) - 2018 Remaster   
4                  Joji                      SLOW DANCING IN THE DARK   
...                 ...                                           ...   
9389                NaN                                           NaN   
9390                NaN                                           NaN   
9391                NaN                                           NaN   
9392                NaN                                           NaN   
9393                NaN                                           NaN   

                                                   tags  
0     cloud rap, emo rap, Juice WRLD, MySpotigramBot...  
1      


Total time elapsed: 00:00:34


In [ ]:

for row_id in df.index:     
    if len(df.loc[row_id,'tags'])==0:
        df = df.drop(row_id)

In [ ]:

happyTags = "cheerful, cheer up, festive, jolly, jovial, merry, cheer, cheering, \
cheery, get happy, rejoice, songs that are cheerful, sunny, happiness, happy songs, happy music, glad, mood: happy, \
upbeat, gleeful,  \
ardor, stimulating, thrilling, titillating, joyful, content, blissful, euphoric, carefree, lighthearted, exuberant, playful, \
delightful, optimistic, lively, positive"
sadTags = "sad, sadness, unhappy,feeling sad, mood: sad - slightly, sad song, \
depressed, blue, dark, depressive, dreary, gloom, darkness, depress, depression, depressing, \
grief, heartbreak, mournful, sorrow, sorry, doleful, heartache, heartbreaking, heartsick, lachrymose, mourning, \
plaintive, regret, sorrowful, tearful, forlorn, downcast, heartbroken, \
downhearted, heavy-hearted, bitter, lamenting, hopeless"
calmTags = "peace,melancholic,slightly, melancholy,gloomy,zen, quiet, ease, soft, light, serene, smooth, nature, gentle, warm, rest, flow, dream, pure,\
glow, mellow, bliss, tranquil, clear, relax, silent, airy, cool, lofi, mist, meadow, water, drift, forest, green, sky, cloud, deep, breathe, peace, \
vibe, clear, meditative, solace, quietude, restore, calm, unwind, soft, lake, sound, wave, breeze, morning, dawn, dusk, slow, blue, mellow, grounded,\
centering, breeze, sunrise, sunset, solitude, soothing, chill, tranquil, zenful, rooted, nature, clear, uncluttered, easeful, harmony, healing, reflective,\
silence, space, dusk, twilight, mild, melodic, soft-spoken, restful, serene, airy, sky, journey, cleansing, cocoon, meditative, warmth, breath, introspection,\
lull, pure, weightless, dreamlike, despondent, somber, wistful, lonely,cushion, balance, haven, light, wander, lofi, moonlight, grounding, cloud, soundscape, timeless, placid, tender, purity,\
mindful, breathe, peaceful, solace, cool, zen, meditative, restful, calm, decompress, breathe, placidity, chill, pacify, slumber, sanctuary, slumber, harmonic,\
melody, cloud, meditative, starlit, flow, dusk, meadow, breeze, unwind, space, reflective, cushion, embrace, nest, soothe, feather, weightless, stillness, harmony,\
grounding, refresh, unwind, silence, airy, breeze, unwind, celestial, dawn, cooling, ease, breath, peaceful, lightness, soothing, dusk, melody, blanket, breath, tranquility, solitude, cloud, hug, float, mellow, quiet, reflective, tender, cocoon, zen, lofi, drift, meditative, simplicity, feather, blanketed, song, nature, gentle, nourish, horizon, blue, sunrise, quiet, placid, blanket, rest, oasis, unwind, moon, flow, balanced, breeze, dawn, cooling, water, zenful, slow, harmony, cocoon, nature, zen, slow, breeze, dawn, light, calmness, drift, tender, calm, soft, tender, introspection, mellow, sky, lofi, gentle, cloud, calmness, quiet, stillness, tranquility, solitude, breathe, breeze, unwind, night, quiet, zenful, blue, cocoon, oasis, soft, gentle, sunrise, slow, placid, soft, silence, oasis, unwind, breeze, journey, purity, slow, quiet, tender, pure, cocoon, simplicity, drift, ease, solitude, rest, dusk, horizon, slumber, breathe, dawn, gentle, pure, silent, oasis, unwind, blanket, smooth, cool, dawn, float, peaceful, cocoon, dawn, dawn, dawn, cocoon, mellow, quiet, nest, cooling, peaceful, pure, pure, pure, silence, serenity, dawn, flow, oasis, breath, purity, soundscape, pure, morning, pure, warmth, pure, introspection, breath, pure, cocoon, peace, morning, oasis, pure, dream, soft, tranquility, sky, pure, flow, pure, zen, pure, unwind, breath, blanketed, pure, introspection, purity, breath, cool, pure, pure, oasis, silence, soundscape, meditative, peace, calmness, tranquility, serene, slumber, blanketed, nest, peaceful, ease, calm, peace, peace, pure, peace, peace, serene, peace, soft, soft, soft, purity, pure, pure, quiet, soft, soft, soft, soft, quiet, sound, silence, calmness, peace, pure, dawn, silence, sound, purity, tranquility, dawn, dawn, blanketed, calmness, peace, peace, morning, morning, morning, morning, silence, silence, silence, slumber, slumber, slumber,dawn"
energeticTags = " high spirits, zest, enthusiastic, buoyancy, elation,mood: upbeat, excitement, exciting, exhilarating, thrill,lively, vibrant, dynamic, spirited, enthusiastic, zesty, high-energy, active, invigorating, bouncy, \
excited, pop, catchy, fast-paced, electric, passionate, forceful, peppy, driven, bold, quick, animated, vigorous, \
positive, assertive, tireless, fierce, spunky, motivating, burning, intense, pulsating, powerful, kickstart, explosive, \
zestful, full-throttle, feverish, hot-blooded, unstoppable, go-getter, enterprising, dazzling, hyper, eager, fast-moving, \
forward-moving, full-speed, high-voltage, uplifting, vivacious, fun-filled, hustling, bustling, full-of-beans, action-packed, \
thrumming, sharp, speedy, quick-footed, frisky, ready, lit, determined, bright, hyperactive, swift, strong-willed, sprinting, \
motion-filled, running, action-driven, swift-footed, bold-hearted, non-stop, bubbly, firecracker, empowered, sprightly, \
stirring, fuelled, rapid, responsive, rushing, jumpy, quick-tempo, full-speed, laser-focused, racing, buzzing, proactive, \
turbocharged, relentless, excitable, recharged, hypercharged, busy, energized, thriving, pumping, stimulating, pushing, \
rapid-paced, overdrive, motivating, ultra, empowered, workhorse, gung-ho, red-hot, inspiring, bounding, untiring, thumping, \
electrifying, sporty, frenzied, full-pull, shootout, driving, laser-focused, motivating, pumped-up, restless, all-in, glowing, \
fast, sharp-witted, accelerating, kinetic, explosive, exciting, forward-thinking, high-octane, challenging, exhilarating, \
pumped, forward-looking, high-spirited, supercharged, chasing, shooting, proactive, empowering, strong, engaging, high-speed, \
fast-moving, charged, racing, stimulating, vital, swift, high-voltage, boosting, bustling, rushing, vivacious, spry, \
turbo-charged, electrified, action-packed, recharged, turbo, vigorous, empowering, highly-energetic, animated, pushy, fervent, \
fired-up, up-tempo, athletic, gritty, unleashed, go-ahead, rousing, electrifying, get-going, stormy, storming, breathtaking, \
ripping, daring, bombastic, fierce-souled, red-blooded, action-filled, epic, showtime, spirited, courageous, full-tilt, \
fire-starting, trailblazing, ambition-driven, super-active, take-off, on-fire, heated, charge-up, adrenaline-rush, sparking, \
competitive, action-heavy, race-speed, hotfooted, sparking, heroic, lightning-fast, brisk, fire-driven, rowdy, get-set, \
hard-hitting, move-maker, intrepid, game-on, keen, mobilized, lionhearted, peak-performing, go-getting, hurly-burly, \
stimulated, daredevil, electric-current, go-mode, no-nonsense, determined-stride, headstrong, stamina-filled, adventurous, \
sizzler, swagger, nitro-fueled, relentless-drive, turbo-speed, forceful, relentless-rhythm, with-a-vengeance, punchy, \
charge-ahead, soaring, feel-the-burn, accelerated, force-in-motion, hit-the-deck, headlong, clear-the-way, energy-to-spare, \
fly-high, in-the-zone, take-charge, zestful-rhythm, push-the-limits, rally, hard-boiled, unrelenting, get-a-move-on, spirited-pace, \
race-ready, hardcore, push-boundaries, dare-all, top-speed, feel-the-heat, unleash, no-brakes, warrior-spirit, bold-force, \
limitless, sparking-energy, no-rest, one-more-round, thunderous, zoom-ahead, breakout, above-and-beyond, off-the-charts, \
leaping, mega-charged, storm-force, blast-off, ready-set-go, thunderbolt, relentless-energy, show-of-force, knock-out, \
test-the-limits, hit-the-ground-running, full-effort, strain-the-muscle, burst-of-energy, energized-pace, fly-faster, \
foot-to-the-floor, no-limits, max-out, overdrive-momentum, endless, battle-ready, zero-to-sixty, lion-pace, action-hero, \
smash-the-goals, dead-run, bulldoze, spirited-march, the-extra-mile, relentless-push, ultra-determined, ground-breaking, \
pushing-the-edge, rocket-force, lit-up, fever-pitch, super-force, ultra-intense, just-do-it, rise-and-shine, fortitude, \
force-beyond, bursting-with-energy, heat-up, quick-start, unwavering, get-hustling, speed-it-up, next-level, ultra-focus, \
rallying, all-in, jump-start, beast-mode, fire-up, unstoppable-force, locked-in, limitless-potential, full-out, turbo-thrust, \
zero-in, sky-high, up-and-at-em, ride-the-wave, surge, hot-wired, lightspeed, on-a-roll, max-potential, all-geared-up, \
breakneck, game-face, strength-driven, no-holds-barred, fast-track, power-rush, in-the-lead, 200-percent, on-fire, epic-speed, \
lift-off, rapid-fire, lights-out, speed-run, ignite, turbo-burst, zoom, time-to-shine, amp-up, non-stop-move, action-rush, \
momentum, non-stop-energy, beast-drive, ultra-speed, heart-pumping, fired, ignited, high-caliber, break-boundaries, \
light-the-spark, turbo-power, power-move, peak, seize-the-moment, lightning-speed, revved-up, max-effort, extra-mile, \
in-the-lead, go-further, stand-out, power-pack, one-way-up, never-stop, max-charge, pumped-energy, big-league, high-achiever, \
top-notch, fast-action, extra-strength, go-faster, quick-boost, ultra-run, hustle, slam, play-hard, go-beyond, above-par, \
hardcore-rush, non-stop-strive, fueled-up, on-the-go, up-level, rock-and-roll, surge-forward, max-output, relentless-pace, \
pushing-the-bar, hustle-and-grind, unbreakable, driven-force, punch-the-clock, on-the-move, warp-speed, take-the-lead, \
beating-the-odds, ultra-fast, outdo-yourself, no-halt, peak-power, skyward-bound, goal-driven, leap-ahead, beyond-limits, \
fast-forward, show-stopper, highly-ambitious, conquer-all, straight-ahead, top-gear, all-guts, foot-on-the-pedal, unyielding, \
no-stopping, rise-and-run, life-in-the-fast-lane, clear-skies-ahead, go-go-go, run-with-it, step-up, blast-into-action, \
ahead-of-the-pack, heavy-hitter, supreme-force, super-sprint, go-further, blow-past, extreme-velocity, headstrong, top-speed, \
wild-ride, no-sleep, endurance, mad-dash, relentless-spirit, force-of-nature, extra-power, ready-to-go, action-force, \
super-charged, leap-of-faith, zoom-ahead, leap-in, rally-up, thrive, unflagging, bold-move, hit-the-road, power-drive, \
shout-out, make-it-happen, no-hesitation, do-or-die, resilient, fire-it-up, push-forward, top-of-the-line, above-and-beyond, \
turn-it-up, hard-charger, mind-over-matter, rise-up, shock-and-awe, make-waves, break-the-mold, bolt-ahead, fearless, \
mega-thrust, onward, a-game, strive-ahead, rolling, get-set, peak-performance, turbo-blast, leave-no-doubt, unstoppable-spirit, \
bring-the-heat, accelerate, bold-pace, make-it-count, the-edge, elite-speed, get-up-and-go, max-speed, quick-hit, cut-loose, \
roaring, hyper-focus, hands-on, make-it-big, race-the-clock, bring-it-on, action-starter, go-the-distance, no-hold-back, \
peak-stride, steamroll, charged-up, burning-pace, hit-the-pace, stamina-driven, step-up-speed, rocket-ahead, all-guts, \
barrel-ahead, clear-path, no-boundaries, on-pace, rush-for-it, charge-ahead, spearhead, ambitious-power, on-point, \
master-the-moment, invincible, energized-stride, max-thrust, supercharged-speed, grit, high-adrenaline, full-fury, all-set, \
full-torque, no-downshift, keep-the-pace, drive-power, go-all-out, red-line, stay-ahead, maximize, nitrous, on-a-mission, \
rapid-response, driven-spirit, engage, break-through, lift-off-momentum, power-on, hands-down, high-velocity, no-slow-down, \
reach-the-top, epic-force, full-potential, driven-force, take-the-wheel, shot-of-energy, high-impact, limitless-drive, \
force-to-reckon, full-bore, get-the-edge, super-fast, show-the-world, hot-on-the-heels, blow-past, overclock, race-ready, \
outpace, cutting-edge, lights-out, nitro-speed, action-wave, forge-ahead, fire-the-boost, extra-mile-power, game-on, \
quick-react, relentless-heart, ultra-focus, momentum-builder, skyward-drift, and-then-some, driven-fuel, bold-outlook, \
show-up, ultra-fast-move, full-out-stride, beyond-ready, fuel-the-flame, intensity, mega-charge, ace-the-goal, pushing-the-envelop, \
stoke-the-fire, seize-the-chance, ace-the-test, peak-the-meter, next-level-move, in-the-fast-lane, pull-ahead, jump-the-gun, \
ultra-move, break-the-limit, all-out-stride, get-up, all-systems-go, go-at-it, full-tilt-stride, move-the-meter, beyond-control, \
put-to-the-test, power-zone, star-power, peak-challenge, master-the-moment, wild-energy, stay-strong, fast-force, roll-with-it, \
make-it-fast, all-fuel, strike-ahead, come-out-on-top, beyond-speed, aim-high, hard-drive, full-throttle-ahead, forward-motion, \
command-the-day, charge-the-field, make-it-count, adrenaline-packed, speed-evolve, amp-the-power, start-your-engines, race-the-day"


# Convert all tag lists to lowercase lists
happyTags = happyTags.lower().split(", ")
sadTags = sadTags.lower().split(", ")
calmTags = calmTags.lower().split(", ")
energeticTags = energeticTags.lower().split(", ")

### Based on the tag numbers from sad group or the happy group, we can assign a mood value 1(happy) or 0(sad) to the mood column 

In [ ]:
print(df.columns)  # Check if 'tags' is a valid column name
print(df['tags'].head())  # Check the first few rows of the 'tags' column

In [ ]:
# Define the mood assignment function
def assign_mood(tags):
    if not tags or tags == 'No tags found':  # Handle empty or invalid tags
        return None

    # Ensure the tags are properly formatted and split
    tag_list = [tag.strip().lower() for tag in tags.split(',')]
    
    # Check for mood categories
    if any(tag in happyTags for tag in tag_list):
        return 'happy'
    elif any(tag in sadTags for tag in tag_list):
        return 'sad'
    elif any(tag in calmTags for tag in tag_list):
        return 'calm'
    elif any(tag in energeticTags for tag in tag_list):
        return 'energetic'
   
    else:
        return 'no match'  # Use a placeholder for unmatched tags for debugging

In [ ]:
# Print the column names to verify
print("Available columns:", df.columns)

# Apply the updated mood assignment function
df['mood'] = df['tags'].apply(assign_mood)

# Filter rows where no valid mood was assigned
df = df[df['mood'] != 'no match']
# Display all song indices along with their moods
print(df[['artists', 'track_name', 'mood']])

# Optionally, save the updated DataFrame to a CSV file
df.to_csv("C:\\Users\\sunitha\\Downloads\\updated_songs_with_moods.csv", index=False)

# Verify the result
print("Songs with their moods:")
print(df[['artists', 'track_name', 'mood']].head())

Available columns: Index(['track_id', 'artists', 'track_name', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'lyrics', 'tags'],
      dtype='object')
                artists                                    track_name  mood
1     Schoolgirl Byebye                                     Year,2015  None
2            Juice WRLD                                  Lucid Dreams  calm
3         Fleetwood Mac  Rhiannon (Will You Ever Win) - 2018 Remaster  None
4                  Joji                      SLOW DANCING IN THE DARK   sad
5              Revenant                                     Year 2018  None
...                 ...                                           ...   ...
9389                NaN                                           NaN  None
9390                NaN                                           NaN  None
9391                NaN                                

In [ ]:
df['mood']
df = df.dropna(subset=['mood'])
# Print index and mood values
for index, mood in df['mood'].items():
    print(f"Song {index}: Mood {mood}")

Song 2: Mood calm
Song 4: Mood sad
Song 6: Mood energetic
Song 10: Mood sad
Song 12: Mood calm
Song 14: Mood sad
Song 24: Mood energetic
Song 28: Mood energetic
Song 29: Mood energetic
Song 34: Mood sad
Song 42: Mood energetic
Song 51: Mood energetic
Song 53: Mood calm
Song 61: Mood sad
Song 62: Mood calm
Song 63: Mood energetic
Song 65: Mood energetic
Song 71: Mood energetic
Song 75: Mood energetic
Song 77: Mood energetic
Song 84: Mood energetic
Song 85: Mood energetic
Song 88: Mood energetic
Song 91: Mood sad
Song 95: Mood energetic
Song 97: Mood sad
Song 99: Mood energetic
Song 101: Mood energetic
Song 105: Mood energetic
Song 109: Mood energetic
Song 111: Mood energetic
Song 119: Mood energetic
Song 121: Mood energetic
Song 127: Mood energetic
Song 135: Mood calm
Song 145: Mood energetic
Song 150: Mood energetic
Song 151: Mood energetic
Song 153: Mood energetic
Song 157: Mood energetic
Song 159: Mood sad
Song 161: Mood energetic
Song 163: Mood happy
Song 171: Mood energetic
Song 17

In [ ]:
# Save final balanced data
if not df.empty:
    df.to_csv('lyrics_166.csv', index=False, encoding='utf-8')
else:
    print("No data to save.")